In [7]:
import gmaps


In [8]:
import numpy as np
import pandas as pd
import requests
import time
# Dependencies
import requests
import json
# Google developer API key
from config import gkey
# Retrieve Google API key from config.py
from uszipcode import SearchEngine
import matplotlib.pyplot as plt
import time
marijuana_df=pd.read_csv('CannabisLicData_cleaned.csv')
marijuana_zip=marijuana_df['Zip Code'].value_counts()
marijuana_zip.sort_index(inplace=True)
search = SearchEngine(simple_zipcode=True) # set simple_zipcode=False to use rich info database


zipCodes=pd.read_csv('laZipCodes.csv', encoding = "ISO-8859-1")
zipCodes.head()
marijuana_zipDataFrame = marijuana_zip.to_frame().reset_index()
marijuana_zipDataFrame.columns = ['zip', 'count']


dataSet = []
for zipCode in marijuana_zipDataFrame["zip"] :
    zipcodeData = search.by_zipcode(zipCode).to_dict()
    dataSet.append([zipCode,zipcodeData.get("lat"),zipcodeData.get("lng")])

zipCodesData = pd.DataFrame(dataSet)
zipCodesData.columns = ['zip', 'lat','lng']


In [9]:
newData = pd.merge(zipCodesData, marijuana_zipDataFrame, on='zip', how='outer')
newData.dropna(inplace=True)

In [10]:
locations = newData[["lat", "lng"]].astype(float)
poverty_rate = newData["count"].astype(float)


In [12]:
gmaps.configure(api_key=gkey)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 10
heat_layer.point_radius = 1

fig.add_layer(heat_layer)
